# ngrokのインストール
Google ColabでTensorBoardを利用するために、ローカルPCとの接続をトンネリングするツールをインストール。

In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-11-11 05:51:29--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.214.163.243, 3.223.119.4, 52.204.140.35, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.214.163.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  14.1MB/s    in 0.9s    

2019-11-11 05:51:31 (14.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


# TensorBoardをバックグラウンドで実行

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

# ngrokバックグラウンドプロセスを起動
TensorBoardのポート6006を開放する。

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

# TensorBoardのURLを取得

In [4]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://b4dedf1a.ngrok.io


# KerasのCNNモデルを実行

In [5]:
import numpy as np
import keras
from keras import Input, layers
from keras.layers import Dense, Embedding, LSTM, concatenate, Activation
from keras.layers import Flatten, Conv2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Sequential, Model
from keras.utils.np_utils import to_categorical

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [0]:
from keras.datasets import cifar10

In [7]:
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [0]:
#画像を0-1の範囲で正規化
x_train = x_train.astype('float32')/255.0
x_test  = x_test.astype('float32')/255.0

#正解ラベルをOne-Hot表現に変換
y_train = to_categorical(y_train,10)
y_test  = to_categorical(y_test,10)

In [9]:
#モデルを構築
model=Sequential()

model.add(Conv2D(32,(3,3),padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
# モデルの要約を表示
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)       

In [11]:
# モデルのコンパイル
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
# 訓練中に適応するコールバックを作成
callbacks = [keras.callbacks.TensorBoard(
    log_dir='./log', 
    histogram_freq=1,
    write_graph=True, 
    write_images=True,
    #embeddings_freq=1
    )
]

In [13]:
# モデルの訓練と評価
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 50000 samples, validate on 10000 samples









Epoch 1/10
50000/50000 [==============================] - 9s 171us/step - loss: 1.6276 - acc: 0.4019 - val_loss: 1.2750 - val_acc: 0.5440

Epoch 2/10
50000/50000 [==============================] - 5s 92us/step - loss: 1.2140 - acc: 0.5662 - val_loss: 1.0357 - val_acc: 0.6287
Epoch 3/10
50000/50000 [==============================] - 5s 92us/step - loss: 1.0324 - acc: 0.6335 - val_loss: 0.9019 - val_acc: 0.6791
Epoch 4/10
50000/50000 [==============================] - 5s 95us/step - loss: 0.9210 - acc: 0.6754 - val_loss: 0.8596 - val_acc: 0.6987
Epoch 5/10
50000/50000 [==============================] - 5s 93us/step - loss: 0.8323 - acc: 0.7057 - val_loss: 0.7713 - val_acc: 0.7328
Epoch 6/10
50000/50000 [==============================] - 5s 93us/step - loss: 0.7613 - acc: 0.7336 - val_loss: 0.7447 - val_acc: 0.7370
Epoch 7/10
50000/500

In [0]:
# 損失値と正解率を取得
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
# 損失値をグラフ表示
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# 正解率をグラフ表示
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()